# Analyze the sharded dataset in memory

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import anndata as ad

In [ ]:
ln.track()

In [ ]:
ln.Dataset.filter().df()

In [ ]:
dataset = ln.Dataset.filter(name="My versioned scRNA-seq dataset", version="2").one()

In [ ]:
dataset.files.df()

If the dataset doesn't consist of too many files, we can now load it into memory.

Under-the-hood, the `AnnData` objects are concatenated during loading.

The amount of time this takes depends on a variety of factors.

If it occurs often, one might consider storing a concatenated version of the dataset, rather than the individual pieces.

In [ ]:
adata = dataset.load()

The default is an outer join during concatenation as in pandas:

In [ ]:
adata

The `AnnData` has the reference to the individual files in the `.obs` annotations:

In [ ]:
adata.obs.file_id.cat.categories

We can easily obtain ensemble IDs for gene symbols using the look up object:

In [ ]:
genes = lb.Gene.lookup(field="symbol")

In [ ]:
genes.itm2a.ensembl_gene_id

Let us create a plot:

In [ ]:
import scanpy as sc

sc.pp.pca(adata, n_comps=2)

In [ ]:
sc.pl.pca(
    adata,
    color=genes.itm2a.ensembl_gene_id,
    title=(
        f"{genes.itm2a.symbol} / {genes.itm2a.ensembl_gene_id} /"
        f" {genes.itm2a.description}"
    ),
    save="_itm2a",
)

In [ ]:
file = ln.File("./figures/pca_itm2a.pdf", description="My result on ITM2A")

In [ ]:
file.save()

In [ ]:
file.view_flow()

In [ ]:
# clean up test instance
!lamin delete --force test-scrna
!rm -r ./test-scrna